In [1]:
import pandas as pd

In [2]:
import math

In [3]:
from time import time 

In [4]:
from array import *

In [5]:
import unicodedata

In [6]:
from haversine import haversine, Unit

In [7]:
tiempoInicial=time()

In [8]:
datos = pd.read_excel("datos.xlsx")

In [9]:
def velocidad(vehiculo):
    if str(vehiculo)=="motorcycle":
        return (20/3600)
    elif str(vehiculo)=="car":
        return (17/3600)
    elif str(vehiculo)=="bicycle":
        return (12/3600)
    elif str(vehiculo)=="walking":
        return (3/3600)

In [10]:
#conectar rappis con ordenes que pueden cumplir
Costo={}
CostoTienda={}
Instancia={}
courierNodos=set()
pedidosNodos=set()
for i in datos.index:
    
    locCourier=(datos.loc[i, "courier_lat"],datos.loc[i,"courier_lng"])
    locStore=(datos.loc[i,"store_lat"],datos.loc[i,"store_lng"])
    locUser=(datos.loc[i,"user_lat"],datos.loc[i,"user_lng"])
    vel= velocidad(str(datos.loc[i,"vehicle_type"]))
    maxStore= datos.loc[i,"store_eta_ub"]
    maxUser=datos.loc[i,"user_eta_ub"]
    StoreService = datos.loc[i,"store_service_time"]
    tiempoHastaTienda=(haversine(locCourier,locStore))/vel
    tiempoTiendaUsuario= StoreService + haversine(locStore,locUser)/vel
    #verificar si realizar esta orden por este rappi es factible
    #podemos disminuir el tiempo minimo de cumplimiento
    if (tiempoHastaTienda) <= maxStore and tiempoHastaTienda+ tiempoTiendaUsuario <= maxUser: 
        if(tiempoHastaTienda+ tiempoTiendaUsuario < 3000):
            Costo[datos.loc[i,"courier_id"],datos.loc[i,"order_id"]]=tiempoHastaTienda+ tiempoTiendaUsuario
            CostoTienda[datos.loc[i,"courier_id"],datos.loc[i,"order_id"]]=tiempoHastaTienda
            Instancia[datos.loc[i,"courier_id"],datos.loc[i,"order_id"]]= datos.loc[i,"instance_id"]
            
            courierNodos.add(datos.loc[i,"courier_id"])
            pedidosNodos.add(datos.loc[i,"order_id"])
        
    

In [11]:
nodoOferta="oferta"
nodoDemanda="demanda"

b={}

for i in courierNodos:
    Costo[nodoOferta,i]=0
    CostoTienda[nodoOferta,i]=0
    b[i]=0

for j in pedidosNodos:
    Costo[j,nodoDemanda]=0
    CostoTienda[j,nodoDemanda]=0
    b[j]=0


    
b[nodoOferta]=0
b[nodoDemanda]=0




In [12]:
A=list(Costo.keys())



In [13]:
N=list(b.keys())


In [14]:
total=7159

In [15]:
from gurobipy import * 

m = Model("Nombre")
m

Using license file C:\Users\Javier Fernandez\gurobi.lic
Academic license - for non-commercial use only


<gurobi.Model Continuous instance Nombre: 0 constrs, 0 vars, No parameter changes>

In [16]:
x={} #variables
z=m.addVar(vtype=GRB.INTEGER, name="z")


for (i,j) in A:
    x[i,j]=m.addVar(vtype = GRB.INTEGER, name = "x_"+str( (i,j) ),ub=1)
m.update()

In [17]:
m.setObjective((total-z)*3600+ sum(x[k,j]*Costo[k,j] for (k,j) in A), GRB.MINIMIZE)
for i in N:
    if( i == nodoOferta):
        m.addConstr( sum(x[k,j] for (k,j) in A if k==i) - sum(x[j,k] for (j,k) in A if k==i) -z == 0 )
    elif (i==nodoDemanda):
        m.addConstr( sum(x[k,j] for (k,j) in A if k==i) - sum(x[j,k] for (j,k) in A if k==i) + z== 0 )
    else:
        m.addConstr( sum(x[k,j] for (k,j) in A if k==i) - sum(x[j,k] for (j,k) in A if k==i) == 0 )

m.setParam("OutputFlag", 0) # gurobi que no hable carreta
m.optimize()

In [18]:
tiempofinal=time()
print(tiempofinal-tiempoInicial)

21977.86793756485


In [19]:

print("instancia"+"orden"+","+"Rappi"+","+"tiempo de llegada restaurante"+","+ "tiempo de llegada cliente")

for (i,j) in A:
     if x[i,j].x > 0 and i!=nodoOferta and j != nodoDemanda:
        print(str(Instancia[i,j])+","+str(j)+","+str(i)+","+str(CostoTienda[i,j])+","+str(Costo[i,j]))
    
    

instanciaorden,Rappi,tiempo de llegada restaurante,tiempo de llegada cliente
b42ca81f3,102029070,163444,71.96194176844301,522.3133962587049
b42ca81f3,102029476,302659,32.462647462802394,357.3134855350125
b42ca81f3,102030655,394816,96.73009131824752,622.7203437194618
b42ca81f3,102030862,272444,11.486104767298768,1003.4753531422208
b42ca81f3,102030976,80861,110.6686878414606,1177.0203808538563
b42ca81f3,102031227,38953,45.008895021348245,515.5339672452145
b42ca81f3,102031284,321578,98.46340861826734,592.3036434766906
b42ca81f3,102031584,280399,82.6534640392673,1054.2382328193403
b42ca81f3,102031606,252081,80.37207005661207,589.061587004054
b42ca81f3,102031764,209478,78.3164590974668,519.4091533622545
b42ca81f3,102031807,376223,41.65070914744488,594.252204037536
b42ca81f3,102031970,83891,264.6691822322701,1171.2608855225405
b42ca81f3,102032059,198667,39.76283579037609,343.4342863181185
b42ca81f3,102032166,171951,57.33959517398603,787.0123083719284
b42ca81f3,102032237,245762,87.13695624776

b42ca81f3,102036565,34235,192.31507997177278,811.4447838004038
b42ca81f3,102036582,282870,261.9882767363757,747.5288019329062
b42ca81f3,102036634,182800,40.800149620185714,850.3570188388752
b42ca81f3,102036650,312699,66.01712188681562,540.1842645956597
b42ca81f3,102036658,196447,142.1512622855978,967.1084485175057
b42ca81f3,102036672,315107,458.2657684902062,732.728931851424
b42ca81f3,102036694,263913,244.3728900699456,954.1261630207349
b42ca81f3,102036710,290096,113.27026835270061,574.463738251221
b42ca81f3,102036728,297299,40.482700432080584,819.4632169127473
b42ca81f3,102036755,298200,297.6077447338949,1117.4924011203962
b42ca81f3,102036781,307003,29.754439053625294,713.2134512013232
b42ca81f3,102036782,228060,108.13949064831084,1061.2504054393075
b42ca81f3,102036793,76581,1.9622584537574355,1033.338102946076
b42ca81f3,102036806,258425,111.96256329995802,1039.379621622861
b42ca81f3,102036864,158843,51.9440413559526,1188.1434830570208
b42ca81f3,102036890,275498,116.16788509148527,671

194ce5835,102912257,211932,3.561167430903795,525.2542470392694
194ce5835,102912269,296839,34.360688821188745,469.7847105828191
194ce5835,102912540,338090,8.589241286423661,856.474700072577
194ce5835,102912766,377720,7.674841787148909,387.7195481025442
194ce5835,102913157,225257,28.4683612338366,569.0934047367119
194ce5835,102913515,249364,7.653215818186487,350.79701160607897
194ce5835,102913699,346086,12.88705655158356,670.7298049182208
194ce5835,102913740,211044,11.46873610749672,379.2589127601394
194ce5835,102913913,165214,27.704188659408516,300.45632784615486
194ce5835,102913953,298964,1.4996845535445575,136.79143945323023
194ce5835,102914219,318048,37.83419239889422,412.4978588090624
194ce5835,102914301,120655,83.38874773402308,746.5311436628286
194ce5835,102914421,273398,22.149712953057207,572.3980702919123
194ce5835,102914445,365313,2.8879388986420236,215.19254958738392
194ce5835,102914929,116280,44.96445901215032,379.9099851388462
194ce5835,102915471,289176,6.140454190571935,478

19df4db80,102919468,313412,58.594847172308455,467.451723178123
19df4db80,102919527,167718,49.21435557050254,593.7549603950343
19df4db80,102919586,192142,36.36058839764294,594.0189118945857
19df4db80,102919588,78432,6.132076554801845,769.3382541389076
19df4db80,102919595,306886,27.510689819863444,457.6258214341527
19df4db80,102919598,258753,11.902991121920971,808.9306985902168
19df4db80,102919604,49661,49.03851006272865,577.1135521923717
19df4db80,102919623,320414,44.29963828262486,497.6453057721439
19df4db80,102919625,378743,27.540968020779847,520.0321777970928
19df4db80,102919660,394228,33.25906939655236,814.5520021947995
19df4db80,102919696,393804,15.077733881173621,461.00576128698606
19df4db80,102919820,202001,36.82366681101902,879.7302126071305
19df4db80,102919865,264170,94.73234291940649,612.3143898788322
19df4db80,102919872,70022,17.92830188504958,465.38543679869144
19df4db80,102919874,255138,70.54511851182646,1017.057600571889
19df4db80,102919900,171886,21.52333897729002,889.164

a1a38409b,99783358,198998,11.390532276412186,775.9926818014917
a1a38409b,99783379,165148,30.40521462415338,746.7506944782281
a1a38409b,99783403,321604,175.352009973726,803.5683859441829
a1a38409b,99783493,228372,24.799649523513665,565.5784697293113
a1a38409b,99783494,197692,132.7121052175485,657.2999630161473
a1a38409b,99783505,266619,51.080181570491774,799.249244280443
a1a38409b,99783507,149643,29.164347147953343,541.3008641441817
a1a38409b,99783535,210613,27.497257732799532,883.132938477888
a1a38409b,99783545,266520,44.52511471552268,522.1555353640538
a1a38409b,99783575,302522,138.0095254153902,931.0381946115331
a1a38409b,99783632,226111,72.93976808119591,752.1550321488843
a1a38409b,99783650,158576,4.2283920565638535,748.0037498886863
a1a38409b,99783655,171245,1.8277644516855087,550.999031092237
a1a38409b,99783807,46449,114.76164487449256,821.4437390552735
a1a38409b,99783832,167052,109.24263283408794,834.0638760253034
a1a38409b,99783902,315538,216.68623338471792,1007.5337912589707
a1

...

624d376c6,100892727,297167,112.37119836634828,515.476176582305
624d376c6,100892733,321572,274.48929621655645,1328.2202285725066
624d376c6,100892743,73062,11.135190170300287,513.0152086689183
624d376c6,100892748,205229,28.79487780229398,429.1328039558249
624d376c6,100892779,197647,41.631010338204156,541.4175582160444
624d376c6,100892816,368937,15.590653246532439,611.9866470815462
624d376c6,100892817,316150,87.56685483035446,520.2092786942819
624d376c6,100892825,192238,176.74985534387767,633.6291279981617
624d376c6,100892888,260075,17.859689289790218,397.1125817250085
624d376c6,100892896,192365,95.87615130371606,525.1056875888517
624d376c6,100892909,227275,156.90466718842768,896.3793006225251
624d376c6,100892911,183283,13.686869218775866,775.8211788722126
624d376c6,100892915,156008,3.8955228007325124,519.9309214088835
624d376c6,100892924,301635,46.41435237967897,529.2901714188245
624d376c6,100892940,210804,17.295707818223672,865.8256139963119
624d376c6,100893000,373639,367.1888996875545,

99a10db5d,100230847,349676,1.3507039373848067,467.07888320630195
99a10db5d,100231733,92667,6.3974182265661454,531.0855327206133
99a10db5d,100235179,203047,41.86579082798012,731.6882687166599
99a10db5d,100235393,202417,9.895817821279483,299.2316041588682
99a10db5d,100235396,277511,13.254482846382613,741.5772210201358
99a10db5d,100236640,158769,24.65434228430857,1027.6226328822263
99a10db5d,100237987,339949,6.887932291182836,574.7176365564429
99a10db5d,100239281,171198,8.051078216271197,772.0276656584938
99a10db5d,100239432,316740,4.465794864787328,532.5131914418143
99a10db5d,100241180,277242,18.73736815429179,687.9531083940333
c0b211663,100219710,68157,11.507738053980397,573.9188761920811
c0b211663,100220562,161933,118.47888314691963,498.460442419282
c0b211663,100220593,381573,26.623388004711476,419.1489059413022
c0b211663,100220788,184548,42.60383423951577,561.8935001040435
c0b211663,100223285,234705,3.5073268681335814,575.2191947919456
c0b211663,100224029,367517,37.79036943307409,421.

c0b211663,100235777,313029,18.051012540878716,682.2453896655562
c0b211663,100235781,322339,133.09092515693033,711.4043021562145
c0b211663,100235798,314474,38.12848781216841,896.8318725878811
c0b211663,100235836,291321,7.004742470615308,976.1765554967554
c0b211663,100235953,218505,7.087602746318739,581.9902681493428
c0b211663,100235992,215721,16.840021500379283,550.1960342566916
c0b211663,100236066,365887,98.16982268174756,942.5285888638318
c0b211663,100236101,101865,4.472752377556293,599.6086378817647
c0b211663,100236110,225606,1.9482525739944851,612.3248543423183
c0b211663,100236118,135920,14.264151926361283,481.56785892009884
c0b211663,100236172,340700,15.807561087088303,563.1562335908485
c0b211663,100236195,110857,4.659242316226273,207.209421945857
c0b211663,100236226,183225,20.638489973738114,383.00792299002126
c0b211663,100236349,369540,7.178009422844628,805.1894290142005
c0b211663,100236402,317795,6.674010243188598,371.0217491744803
c0b211663,100236552,181881,65.43770745041226,60

8350eb5fe,100232599,172490,25.40090286778286,469.8044449406649
8350eb5fe,100232643,383626,113.58674395637354,773.6395116428614
8350eb5fe,100232652,134463,24.575071394418142,242.68143089317985
8350eb5fe,100232669,315858,18.058845751118703,547.5602380113638
8350eb5fe,100232702,286728,27.81699306543512,1035.246707140917
8350eb5fe,100232742,386852,51.069509563096474,402.69259599626525
8350eb5fe,100232749,161630,30.872680294741723,608.5111150394787
8350eb5fe,100232751,199587,46.79162148798118,479.0152614443273
8350eb5fe,100232753,152470,16.0457712449447,876.7395277954106
8350eb5fe,100232810,198415,70.49944718495571,587.2915217163401
8350eb5fe,100232874,171191,25.09627123452581,332.07788616154824
8350eb5fe,100232879,295922,24.012824548819154,899.5955238686951
8350eb5fe,100232896,148982,24.71185483037495,727.8214602801422
8350eb5fe,100232946,267789,21.461754886943474,538.3856185251627
8350eb5fe,100232957,74914,39.4009922053277,542.8275618852149
8350eb5fe,100232964,217214,6.280638319147969,477

8350eb5fe,100235987,280021,3.0635172917396605,166.6050726114346
8350eb5fe,100235990,218776,47.95154260495689,1024.9340407484713
8350eb5fe,100236015,309741,21.62326825676914,464.9936714409347
8350eb5fe,100236017,85660,32.93608880302645,1196.538983348093
8350eb5fe,100236032,167914,47.07785044806958,702.9638148752113
8350eb5fe,100236160,271862,7.52843545831692,556.4876164059414
8350eb5fe,100236181,96680,45.19906859305128,809.2685212993068
8350eb5fe,100236184,191413,37.7701651449276,1028.3067052113981
8350eb5fe,100236200,88148,158.00012351799847,543.5679688588464


In [20]:
print('Obj: %g' % m.objVal)

Obj: 7.781e+06


In [ ]:
#funcion objetivo por instancias 

fo={}


contador=0
valor=0
ins=pd.read_excel("Instancias y Num.xlsx")
for z in ins.index:
    contador=0
    valor=0
    for (i,j) in A:
        if i!=nodoOferta and j != nodoDemanda:
            if ins.loc[z,"instancia"]==str(Instancia[i,j]):
                contador+=1*x[i,j].x
                valor += Costo[i,j]*x[i,j].x
        
        
    valor+=(ins.loc[z,"total"]-contador)*3600
    fo[z]=valor

In [ ]:
print("instancia"+","+"FO")
for z in ins.index:
    print(str(z)+","+str(fo[z]))